In [ ]:
%%HTML
<!-- Mejorar visualización en proyector -->
<style>
.rendered_html {font-size: 1.2em; line-height: 150%;}
div.prompt {min-width: 0ex; padding: 0px;}
.container {width:95% !important;}
</style>

In [ ]:
%autosave 0
%matplotlib notebook
import matplotlib.pyplot as plt
from matplotlib import animation
import numpy as np
import scipy.stats

from IPython.display import display
import ipywidgets as widgets
from functools import partial
slider_layout = widgets.Layout(width='600px', height='20px')
slider_style = {'description_width': 'initial'}
IntSlider_nice = partial(widgets.IntSlider, style=slider_style, layout=slider_layout, continuous_update=False)
FloatSlider_nice = partial(widgets.FloatSlider, style=slider_style, layout=slider_layout, continuous_update=False)
SelSlider_nice = partial(widgets.SelectionSlider, style=slider_style, layout=slider_layout, continuous_update=False)
from scipy.special import erf
gaussian_pdf = lambda x, mu=0, s=1: np.exp(-0.5*(x-mu)**2/s**2)/(s*np.sqrt(2*np.pi))
gaussian_cdf = lambda x, mu=0, s=1: 0.5 + 0.5*erf((x-mu)/(s*np.sqrt(2)))

# Mundo de datos

Los avances tecnológicos recientes nos permiten **medir, almacenar y enviar** datos de toda índole

- Operaciones industriales
- Comercio
- Entretenimiento 
- Datos públicos y gubernamentales
- Datos médicos
- Ciencia: Genómica, Astronomía, Simulaciones, etc
- Vehículos autónomos
- Internet de las cosas

> Los datos crudos tienen poco valor, necesitamos extraer información a partir de los datos

- ¿Cómo se comportan mis datos? ¿Cúales son las observaciones más cómunes? ¿Qué datos son más relevantes?
- ¿Cúal es la diferencia entre dos muestras? ¿Son las diferencias que observo reales o productos del ruido?
- ¿Qué tan probable es que ocurra el suceso $X$? 
- ¿Qué tan arriesgado es tomar la decisión $Y$?

> La **estadística** nos da herramientas para entender los procesos y tomar decisiones


<img src="https://proxy.duckduckgo.com/iu/?u=https%3A%2F%2Fcdn-images-1.medium.com%2Fmax%2F1600%2F1*ufWDxL-5ogd22Rg_37rakw.png&f=1">


En esta clase aprenderemos a usar [`scipy.stats`](https://docs.scipy.org/doc/scipy/reference/stats.html) y [`numpy.random`](https://docs.scipy.org/doc/numpy/reference/routines.random.html) para resolver problemas de **inferencia estadística**

Pero antes, algunos fundamentos:

## Fundamentos: Incerteza

Fuentes de incerteza:
1. Estocasticidad inherente: Sistemas con [dinámicas aleatorias](https://en.wikipedia.org/wiki/Uncertainty_principle)
1. Observación incompleta: Sistema determinista que parece estocástico [cuando no se observa completamente](https://en.wikipedia.org/wiki/Monty_Hall_problem)
1. Modelamiento incompleto: Los [supuestos y aproximaciones del sistema](https://en.wikipedia.org/wiki/Discretization) introducen incerteza 

¿Cómo representamos la incerteza?

> Teoría de probabilidades

Nos proporciona reglas formales para determinar la verosimilitud de una proposición versus otras

## Fundamentos: Variables aleatorias 

- **V.A.** Es una variable que, al ser observada, puede tomar diferentes valores

    - La denotamos como $X$
    
    - Sus observaciones (realizaciones) son $x\sim X$
    
    - Sus realizaciones tienen un dominio $x \in \mathcal{X}$
    
    - La probabilidad de observar $x$ es $P(X=x)$
    
    - Puede ser discreta o continua
    

    
- El comportamiento de $X$ está dictado por 
    - Función de masa de probabilidad (para $X$ discreta)
    
    $P(X=x) \in [0, 1]$
    
    $\sum_{x\in\mathcal{X}} P(X=x) = 1$    
    
    - Función de densidad de probabilidad (para $X$ continua)
    
    $f(x) \geq 0$
    
    $\int_{x\in\mathcal{X}} f(x) \,dx = 1$, 
    
    $P(a\leq X \leq b) = F(b) - F(a) = \int_{a}^{b} f(x) \,dx$
    
        - Función de densidad acumulada: 
        
        $F(a)  = \int_{-\infty}^{a} f(x) \,dx$
        
        
> **Importante:** Sólo en el caso discreto la distribución puede interpretarse como probabilidad

In [ ]:
plt.close('all'); fig, ax = plt.subplots(1, 2, figsize=(8, 3))
dt=1e-4; x = np.arange(-5, 5, step=dt)

def update_plot(x_r):
    xi, xf = x_r
    for axis in ax:
        axis.cla(); 
        axis.set_xlim([-5, 5]);
    ax[0].plot(x, gaussian_pdf(x)); ax[1].plot(x, gaussian_cdf(x));
    xrange = np.arange(xi, xf, step=dt)
    ax[0].fill_between(xrange, 0, gaussian_pdf(xrange), alpha=0.5)
    ax[1].scatter([xi, xf], [gaussian_cdf(xi), gaussian_cdf(xf)], s=100, c='k', zorder=100)
    ax[1].text(xi+0.5, gaussian_cdf(xi), "Init"); ax[1].text(xf+0.5, gaussian_cdf(xf), "End")
    ax[0].set_title("$\int_{x_i}^{x_f} f(x) dx$ = %0.4f" %(np.sum(gaussian_pdf(xrange))*dt))
    area = gaussian_cdf(xf) - gaussian_cdf(xi)
    ax[1].set_title("$F(x_f) - F(x_i)$ = %0.4f" %(area if area >= 0 else 0))

widgets.interact(update_plot, 
         x_r=widgets.FloatRangeSlider(description="$x_i, x_f$", min=-5, max=5, value=[-1, 1]));

## Probabilidad conjunta, marginal y condicional

La probabilidad de dos eventos $X=x$ e $Y=y$ se caracteriza con la distribución conjunta $P(X, Y)$

A partir de la conjunta se pueden obtener la probabilidad marginal de $X$ (o de $Y$)

$$
P(X=x) = \sum_{y \in \mathcal{Y}} P(X=x, Y=y) 
$$

Usando la conjunta y las marginales podemos obtener las probabilidades condicionales

$$
P(Y=y|X=x) = \frac{P(X=x, Y=y)}{P(X=x)}
$$

(ssi $P(X=x) \neq 0$)


In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(8, 4))
ax = fig.add_subplot(1, 2, 1, projection='3d')
x = np.arange(-4, 5, 1)
y = np.arange(-4, 5, 1)
X, Y = np.meshgrid(x, y)
Z = np.zeros_like(X)
Z[-3, 2:-2] = 1
Z[2, 2:-2] = 1
Z[2:-2, 4] = 1
Z = Z/np.sum(Z)

ax.bar(x, np.sum(Z, axis=1), zdir='x', zs=-4)
ax.bar(y, np.sum(Z, axis=0), zdir='y', zs=5)
ax.bar3d(X.ravel(), Y.ravel(), np.zeros_like(Z.ravel()), 1, 1, Z.ravel())
ax.set_xlim([-4, 5]); ax.set_xlabel('X')
ax.set_ylim([-4, 5]); ax.set_ylabel('Y')

ax2 = fig.add_subplot(1, 2, 2)

def update_plot(idx):
    ax2.cla()
    ax2.bar(y, Z[:, idx]/np.sum(Z[:, idx]))
    ax2.set_title("P(Y|X={0})".format(x[idx]))
    ax2.set_ylim([0, 0.55])
    ax2.set_xlim([-4, 4])
    
widgets.interact(update_plot, idx=IntSlider_nice(min=0, max=len(x), value=4));

## Independencia

Si dos V.A. son independientes podemos escribir
$$
\begin{align}
P(x, y)  &= P(x)P(y|x)\nonumber \\
&= P(x)P(y) \nonumber
\end{align}
$$

> Saber que ocurrió $x$ no me sirve da nada para saber si ocurrió $y$
 

Dos V.A. son condicionalmente independientes si

$$
P(x, y|z)  = P(x|z)P(y|z)
$$


## Regla de la cadena

Podemos descomponer una probabilidad conjunta como

$$
\begin{align}
P(x_1, x_2, x_3) &= P(x_3|x_2, x_1) P(x_1, x_2) \nonumber \\
&= P(x_3|x_2, x_1) P(x_2|x_1) P(x_1) \nonumber 
\end{align}
$$

## Ley de probabilidad total

Si el espacio de probabilidad está particionado en $N$ pedazos y se conocen las probabilidades condicionales $P(X=x|Y=y_i)$ podemos calcular la probabilidad del evento $x$ usando

$$
P(X=x) = \sum_{i=1}^N P(X=x|Y=y_i) P(Y=y_i)
$$

## Teorema de Bayes

Podemos escribir 

- la probabilidad de un evento $y$ dado condiciones $x$: $p(y|x)$ 
- en función de nuestro conocimiento *a priori* sobre $y$: $p(y$)
- y de la verosimilitud de que se observen dichas condiciones si $y$ ocurriese: $p(x|y)$

como

$$
P(y | x) = \frac{P(x|y) P(y)}{P(x)} = \frac{P(x|y) P(y)}{\sum_{y\in\mathcal{Y}} P(x|y) P(y)}
$$

## Descriptores de las distribuciones

Podemos describir una variable aleatoria $X$ con $x \sim f(x)$ usando

### Valor esperado

El valor medio de $X$

$$
\mathbb{E}_{x\sim f(x)} [ X ] = \int_{x\in \mathcal{X}} x f(x)  \,dx
$$

### Varianza 

La dispersión de $X$ en torno a su valor medio

$$
\text{Var}[X]  = \mathbb{E}_{x\sim f(x)} \left[\left(X - \mathbb{E}[X] \right)^2 \right]
$$

la relación lineal entre $X$ e $Y$

$$
\text{Cov}[X, Y]  = \mathbb{E}_{x\sim f_X(x), y\sim f_Y(y)} \left[\left(X - \mathbb{E}[X] \right) \left(Y - \mathbb{E}[Y] \right)^T \right]
$$


### Momentos estadísticos

$$
m_k [X] = \mathbb{E}_{x\sim f(x)} [ X^k ]
$$

- Tercer momento: Simetría 
- Cuarto momento: Cúrtosis


In [ ]:
fig, ax = plt.subplots(2, figsize=(7, 3), tight_layout=True)
x = np.linspace(-4, 4, num=1000)

def update_plot(loc, scale, skew, shape):
    distribution1 = scipy.stats.skewnorm(skew, loc=loc, scale=scale)
    distribution2 = scipy.stats.gennorm(shape, loc=loc, scale=scale*np.sqrt(2))
    [ax_.cla() for ax_ in ax]
    ax[0].plot(x, distribution1.pdf(x))
    ax[1].plot(x, distribution2.pdf(x))
    
widgets.interact(update_plot, loc=FloatSlider_nice(min=-3, max=3), 
                 scale=FloatSlider_nice(min=0.01, max=2., value=1), 
                 skew=FloatSlider_nice(min=-5, max=5, value=0),
                 shape=FloatSlider_nice(min=0.1, max=10, value=2));

## Algunas distribuciones de probabilidad

| Distribución | Fenomeno que representa | Ejemplo |
| --- | --- | --- |
| **Bernoulli** | Evento binario  | Lanzamiento de una moneda |
| **Binomial** | Multiples eventos binarios independientes | |
| **Categórica** | Evento con $k$ valores posibles | Lanzamiento de un dado, Ruleta |
| **Poisson** | Conteo de eventos ocurridos en un período de tiempo | Cantidad de alumnos que llegan entre 9:50 y 10:00| 
| **Exponencial** | Valor continuo positivo | Tiempo de espera entre eventos|
| **Gamma** | Valor continuo positivo | Tiempos de espera hasta que ocurren $n$ eventos|
| **Beta** | Valor continuo en $[0, 1]$ |  Tiempo para completar una tarea, proporciones|
| **Normal/Gaussiana** | Valor continuo ubicado en la vecindad de un valor central| [Demasiados](https://galtonboard.com/probabilityexamplesinlife)|
| **Uniforme** | Valor discreto/continuo acotado a un rango, todos con igual probabilidad de ocurrencia| |

<img src="https://thumbs.gfycat.com/AggressiveAromaticBuckeyebutterfly-size_restricted.gif">

***

Podemos usar `np.random` para generar números aleatorios con distintas propiedades
- `randn` : Números reales con distribución normal estándar
- `rand`: Números reales con distribución uniforme en $[0, 1]$
- `randint(low=1, high=10)`: Números enteros con distribución uniforme entre $[0, 10]$

Podemos usar `scipy.stats` para generar datos de una distribución específica
- [continua](https://docs.scipy.org/doc/scipy/reference/stats.html#continuous-distributions)
- [multivariada](https://docs.scipy.org/doc/scipy/reference/stats.html#multivariate-distributions)
- [discreta](https://docs.scipy.org/doc/scipy/reference/stats.html#discrete-distributions)

Las distribuciones tienen un constructor específico y métodos
- `pdf`/`pmf(x)` Retorna la distribución de probabilidad evaluada en $x$
- `cdf(x)` Distribución acumulada evaluada en $x$
- `ppf(p)` Inverso de la distribución acumulada
- `rvs(size=100)` Retorna $100$ muestras a partir de la distribución

## En detalle: Gaussiana/Normal Multivariada

- Dominio $x\in \mathbb{R}^d$
- Parámetros: 
    - Media $\mathbb{E}[X] = \mu \in \mathbb{R}^d$ 
    - Covarianza $\mathbb{E}[(X-\mu)(X-\mu)^T] = \Sigma \in \mathbb{R}^{d\times d}$ (semidefinida positiva)
- Función de densidad de probabilidad
$$
p(x| \mu, \Sigma) = \frac{1}{ \sqrt{(2 \pi)^d} |\Sigma |} \exp \left ( -\frac{1}{2} (x-\mu)^T \Sigma^{-1} (x-\mu) \right)
$$

- Casos especiales
    - Covarianza diagonal: $\Sigma = [\sigma_1^2, \sigma_2^2, \ldots, \sigma_d^2] I$
    - Covarianza isotrópica (esférica): $\Sigma = \sigma^2 I$
    - Normal estándar: $\mu = [0, 0, \ldots, 0]$, $\Sigma = I$

In [ ]:
fig, ax = plt.subplots(1, figsize=(5, 4))

def update_plot(mu1, mu2, s1, s2, rho, seed):
    ax.cla()
    np.random.seed(seed)
    mu = np.array([mu1, mu2]); s = np.diag(np.array([s1, s2]))
    rot_mat = [[np.cos(rho), -np.sin(rho)], [np.sin(rho), np.cos(rho)]]
    L = np.dot(rot_mat, s)
    #x = np.random.multivariate_normal(mean=mu, cov=np.dot(L, L.T), size=5000)
    dist = scipy.stats.multivariate_normal(mean=mu, cov=np.dot(L, L.T))
    x = np.linspace(-3, 3)
    X, Y = np.meshgrid(x, x)
    Z = dist.pdf(np.dstack((X, Y)))
    ax.contour(X, Y, Z)
    xhat = dist.rvs(size=5000)
    ax.scatter(xhat[:, 0], xhat[:, 1], s=5, alpha=0.5)
    ax.set_xlim([-3, 3]); ax.set_ylim([-3, 3])
    ax.set_aspect('equal')
    
widgets.interact(update_plot, mu1=FloatSlider_nice(min=-2, max=2), mu2=FloatSlider_nice(min=-2, max=2),
                 s1=FloatSlider_nice(value=1, min=0.1, max=2), s2=FloatSlider_nice(value=1, min=0.1, max=2),
                 rho=FloatSlider_nice(value=0, min=-np.pi/2, max=np.pi/2), seed=IntSlider_nice());

## Ley de los grandes números

Si $X_1, X_2, \ldots, X_N$ son V.A independientes e idénticamente distribuidas (iid) con media $\mu$  entonces su promedio

$$
\bar X = \frac{1}{N} (X_1 + X_2 + \ldots + X_N)
$$

tiende a $\bar X \to \mu$ cuando $N \to \infty$

## Teorema central del límite

Si $X_1, X_2, \ldots, X_N$ son V.A iid, entonces su promedio 

$$
\bar X \sim \mathcal{N}(\mu, \sigma^2/N)
$$

> Sin importar su distribución original, si sumo muchas VA iid entonces la suma se distribuye normal

## En detalle: Distribución multinomial

- Dominio $x \in \{0, 1, \ldots n\}^k$
- Parámetros: 
    - $n>0$: Número de experimentos 
    - ${p_1, p_2, \ldots, p_k}$: Probabilidad de las categorías donde $\sum_i p_i = 1$
- Función de masa de probabilidad
$$
p(x| m, \{p\}) = \frac{n!}{x_1! x_2! \cdots x_k!} \prod_{i=1}^k {p_i}^{x_i}
$$

- Casos especiales
    - $k = 2$: Distribución binomial
    - $k = 2$ y $n = 1$: Distribución Bernoulli

In [ ]:
dist = scipy.stats.multinomial(n=2, p=[1/6]*6)
dist.rvs(size=1)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 3))

def update_plot(k):
    ax.cla()
    ax.set_title("Promedio de {0} lanzamiento/s de dado".format(k+1))
    dist = scipy.stats.multinomial(n=k+1, p=[1/6]*6)
    repeats = dist.rvs(size=1000)/(k+1)
    average_dice = np.sum(repeats*range(1, 7), axis=1)
    ax.hist(average_dice, bins=12, density=True)
    ax.set_xlim([1, 6])
update_plot(0)
#anim = animation.FuncAnimation(fig, update_plot, frames=20, interval=300, 
#                               repeat=True, blit=False)

# Estadística

La estadística busca:

> Describir fenómenos complejos a partir de observaciones parciales 

> Inferir propiedades de una población basándonos en una muestra

> Usar datos para responder preguntas y tomar decisiones

La estadística es:

> Disciplina científica dedicada al desarrollo y estudio de métodos para recopilar, analizar y extraer información de los datos


## Estadística descriptiva

Sea una muestra de datos. Para entenderla mejor podemos empezar describiendola:
- ¿Discreto o continuo? ¿No-negativo?
- ¿Dónde está localizada? **Media**
- ¿Cuál es su disperción? **Varianza**
- ¿Son las colas iguales o distintas? **Simetría** (*Skewness*)
- ¿Son las colas ligeras o pesadas? **Curtosis** (*Kurtosis*)
- ¿Tiene una moda o múltiples modas?
- ¿Cuantiles? ¿Percentiles? 
- ¿Existen *outliers*?

Podemos responder estas preguntas usando [estadísticos de resumen](https://docs.scipy.org/doc/scipy/reference/stats.html#summary-statistics)

In [ ]:
x = np.linspace(-11, 10, num=1000)
px = 0.7*gaussian_pdf(x, mu=-4, s=2) + 0.3*gaussian_pdf(x, mu=3, s=2)
N = 1000; 
np.random.seed(0)
hatx = np.concatenate((-4 + 2*np.random.randn(int(0.7*N)), 
                       (3 + 2*np.random.randn(int(0.3*N)))))

scipy.stats.describe(hatx)

En ciertos casos podemos responder estas preguntas graficamente usando:

### Histograma

- Es una representación numérica de una distribución
- Nos permite visualizar las características de la distribución
- Se construye dividiendo el dominio en **bines** y contando los datos que caen en cada **bin**
- Está definido por la posición y tamaño de los bines
- Método no-paramétrico para representar distribuciones

In [ ]:
plt.close('all'); fig, ax = plt.subplots(figsize=(7, 4))

def update_plot(nbins): 
    ax.cla()
    ax.plot(x, px, 'k-', linewidth=4, alpha=0.8)
    hist, bin_edges = np.histogram(hatx, bins=nbins, density=True)
    ax.bar(bin_edges[:-1], hist, width=bin_edges[1:] - bin_edges[:-1], 
           edgecolor='k', align='edge', alpha=0.8)
    ax.set_xlabel('x')
    
widgets.interact(update_plot, nbins=SelSlider_nice(options=[2, 5, 10, 15, 20, 50], value=5));

### Kernel density estimation (KDE)

Mismo objetivo que el histograma pero
1. Cada dato es "su propio bin"
1. Los bines se pueden traslapar
1. No se escoge la posición o fronteras de bines, solo su ancho

Para un set de observaciones unidimensionales $\{x_i\}_{i=1,\ldots, N}$ con dominio continuo

$$
\hat f_h(x) = \frac{1}{Nh} \sum_{i=1}^N \kappa \left ( \frac{x - x_i}{h} \right)
$$

donde 
- $h$ es el **ancho de banda del kernel** o **tamaño de kernel**
- $\kappa(u)$ es una **función de kernel** que debe ser positiva, con media cero e integrar a la unidad

Por ejemplo  el famoso kernel Gaussiano

$$
\kappa(u) = \frac{1}{\sqrt{2\pi}} \exp \left ( - \frac{u^2}{2} \right),
$$

Ojo: 
- Usar un kernel gaussiano no es lo mismo que asumir que los datos se distribuyen gaussianos
- Se puede usar un kernel gaussiano en datos que no se distribuyen gaussianos

In [ ]:
from sklearn.neighbors.kde import KernelDensity
plt.close('all'); fig, ax = plt.subplots(figsize=(7, 4))
ax.plot(x, px, 'k-', linewidth=4, alpha=0.8)
ax.scatter(hatx, np.zeros_like(hatx), marker='+', c='k', s=20, alpha=0.1)
ax.set_xlabel('x')
line_kde = ax.plot(x, np.zeros_like(x))
#hs = 0.9*np.std(hatx)*N**(-1/5)
def update(k): 
    kde = scipy.stats.gaussian_kde(hatx, bw_method=lambda kde: k*kde.silverman_factor() )
    line_kde[0].set_ydata(kde.pdf(x))
    
widgets.interact(update, k=SelSlider_nice(description="$k=h/h_s$", options=[1/8, 1/4, 1/2, 1, 2, 4], value=1));

Más opciones de kernels en [`sklearn.neighbors.KernelDensity`](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KernelDensity.html#sklearn.neighbors.KernelDensity)

# Inferencia estadística

> Extraer conclusiones a partir de hechos a través de una premisa científica

- Hechos: Datos
- Premisa: Modelo probabilístico
- Conclusión: Una cantidad no observada que es interesante

> Cuantificar la incerteza de la conclusión dado los datos y el modelo 

Tareas inferenciales
- Ajustar un modelo: **Máxima verosimilitud**
- Verificar el modelo: **Intervalo de confianza**
- Responder una pregunta usando el modelo: **Test de hipótesis**

## Ajuste de modelos paramétricos

Se refieren a aquellos modelos que explicitan una distribución de probabilidad

**Ejemplo:** Queremos ajustar un modelo lineal en sus parámetros a un conjunto de $M$ observaciones ruidosas

Podemos modelar 

$$
y_i = \Phi(x_i) \theta + \varepsilon_i
$$

donde $\varepsilon_i \sim \mathcal{N}(0, \sigma^2)$

es decir que **asumimos** que el ruido es
- Independiente
- Gaussiano
- Con media cero y varianza $\sigma^2$

Con esto podemos escribir la probabilidad de observar $y_i$ dado un cierto valor $\theta$ como

$$
p(y_i|\theta) = \mathcal{N}(\Phi(x_i) \theta, \sigma^2)
$$

La probabilidad de haber observado el conjunto completo es
$$
p(y_1, y_2, \ldots, y_M| \theta)
$$

Podemos **asumir** que los datos son independientes e identicamente distribuidos, en ese caso

$$
\begin{align}
p(y_1, y_2, \ldots, y_M| \theta) &= \prod_{i=1}^M p(y_i|\theta) \nonumber \\
&= \prod_{i=1}^M  \frac{1}{\sqrt{2\pi}\sigma}  \exp \left ( - \frac{1}{2\sigma^2} (y_i - \Phi(x_i) \theta)^2 \right)
\end{align}
$$

***
Llamaremos **verosimilitud** de $\theta$ a 

$$
\mathcal{L}(\theta) = \prod_{i=1}^M p(y_i|\theta) 
$$

y buscamos el $\theta$ más "verosimil" maximizando $\mathcal{L}$ en función de $\theta$
***

Por conveniencia se trabaja con el logaritmo de la verosimilitud. Para el caso anterior

$$
\max_\theta \log \mathcal{L}(\theta) = -\frac{1}{2} \log(2\pi\sigma^2) - \frac{1}{2\sigma^2} \sum_{i=1}^M (y_i - \Phi(x_i) \theta)^2
$$

Por lo tanto 
$$
\min_\theta \log \mathcal{L}(\theta) = \sum_{i=1}^M (y_i - \Phi(x_i) \theta)^2
$$

Que corresponde al problema de **mínimos cuadrados**

¿Qué estamos asumiendo cuando usamos mínimos cuadrados?

¿Qué tan confiable es $\theta$?

# Estimación de máxima verosimilitud

El procedimiento que acabamos de ver se llama *maximum likelihood estimation* (MLE)

Procedimiento

1. Definir los supuestos del problema y el modelo
1. Escribir el logaritmo de la verosimilud de los parámetros $\log \mathcal{L}(\theta)$
1. Encontrar $\theta$ que maximiza 
$$
\hat \theta = \text{arg}\max_\theta \log \mathcal{L}(\theta)
$$


Las distribuciones de  `scipy.stats` tienen los métodos
- `fit (data)` Ajusta una distribución continua a datos usando MLE
- `expect (func)` Valor esperado de una función c/r a la distribución
- `interval (alpha)` Cotas para el intervalo que contiene un porcentaje $\alpha$ de la distribución

Para distribuciones complicadas sin solución analítica se pueden usar métodos iterativos


## Ejemplos 

Observemos la siguiente distribución, ¿Qué características resaltan? ¿Qué distribución sería apropiado ajustar?

In [ ]:
from sklearn import datasets
data_set = datasets.load_breast_cancer()
x, y = data_set['data'][:, 0], data_set['target']
fig, ax = plt.subplots(figsize=(5, 3))
ax.hist(x, bins=20, density=True);

Podemos probar varias distribuciones y observar el resultado

¿Cómo medir la bondad del ajuste?

In [ ]:
x_plot = np.linspace(np.amin(x), np.amax(x), num=50)
dist = scipy.stats.norm
# for dist in [scipy.stats.norm, scipy.stas.lognorm, scipy.stats.beta, scipy.stats.gamma, scipy.stats.uniform]:
params = dist.fit(x)
print(dist.name)
print(params)
p_plot = dist(*params[:-2], loc=params[-2], scale=params[-1]).pdf(x_plot)
ax.plot(x_plot, p_plot, label=dist.name)
plt.legend();

### Bondad de ajuste

Podemos usar el test chi-cuadrado, el [test de Akaike](https://en.wikipedia.org/wiki/Akaike_information_criterion), el test no-paramétrico de Kolmogorov-Smirnov (KS) o gráficos QQ para medir que tan bien se ajusta nuestra distribución teórica a los datos

El test KS nos permite comparar que tan distinta es una distribución continua empírica de una teórica comparando sus CDFs: [`stats.kstest`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kstest.html)

Require que los datos estén estandarizados (media cero y desviación estándar 1)

- Retorna un estadístico: Mientras más cerca a cero, mejor es el ajuste
- También retorna un p-value: Si es menor que $\alpha$ entonces rechazo la hipótesis nula de que las distribuciones son iguales con un $1-\alpha$ de confianza


In [ ]:
x_std = (x-np.mean(x))/np.std(x)
ks_res = []
for dist in [scipy.stats.norm, scipy.stats.lognorm, scipy.stats.beta, scipy.stats.gamma, scipy.stats.uniform]:    
    params = dist.fit(x_std)
    fitted_dist = dist(*params[:-2], loc=params[-2], scale=params[-1])
    ks_res.append(scipy.stats.kstest(rvs=x_std, cdf=fitted_dist.cdf))
    print(dist.name)
    print(ks_res[-1])

# OJO: Hasta acá llegamos hoy

## Ejemplo

Sea el siguiente dataset de consumo de helados per capita

¿Existe correlación entre el consumo de helados y la temperatura?

¿Cómo respondemos esta pregunta usando la formalidad estadística?

Partamos por ajustar un modelo. Asumiendo error gaussiano, muestras iid y un modelo lineal de dos parámetros

$$
y_i = \theta_0 + \theta_1 x_i + \epsilon_i
$$

El estimador de máxima verosimilitud para $\theta$ se obtiene resolviendo

$$
\min_\theta \log \mathcal{L}(\theta) = \sum_{i=1}^M (y_i - \theta_0 - \theta_1 x_i)^2
$$

donde 
$$
\sum_i y_i  - M\theta_0 - \theta_1  \sum_i x_i = 0 \rightarrow \theta_0 = \bar y - \theta_1 \bar x
$$
$$
\sum_i y_i x_i - \theta_0 \sum_i x_i - \theta_1 \sum_i x_i^2 = 0
$$

$$
\theta_1 = \frac{ \sum_i (y_i - \bar y)}{\sum_i (x_i - \bar x)}
$$

In [ ]:
fig, ax = plt.subplots(figsize=(6, 3), tight_layout=True)

temp = np.linspace(10, 30, num=50)
np.random.seed(0)
helados = 0.5*temp - 2 + 1.5*np.random.randn(len(temp))

ax.scatter(temp, helados)
ax.set_xlabel('Temperatura')
ax.set_ylabel('Consumo de helados promedio\nmensual per capita en Chile');

In [ ]:
### Bootstrap resampling



In [ ]:
scipy.stats.norm().fit()

In [ ]:
poly_basis = lambda x,N : np.vstack([x**k for k in range(N)]).T
x = np.linspace(0, 2, num=100)
y = 2*np.cos(2.0*np.pi*x) + np.sin(4.0*np.pi*x) + 0.6*np.random.randn(len(x))
fig, ax = plt.subplots(2, 1, figsize=(6, 4), tight_layout=True, 
                       sharex=True, sharey=True)
ax[0].scatter(x, y, c='k'); ax[1].scatter(x, y, c='k')

T = 10
N = 7
thetas, curves = [], []
for i in range(T):
    idx = np.random.choice(range(len(x)), size=len(x), replace=True)
    thetas.append(np.linalg.lstsq(poly_basis(x[idx], N), y[idx], rcond=None)[0])
    curves.append(np.dot(poly_basis(x, N), thetas[i]))
    ax[0].plot(x, curves[i])
thetas, curves = np.stack(thetas), np.stack(curves)
mu, std = np.mean(curves, axis=0), np.std(curves, axis=0)
ax[1].plot(x, mu, lw=4);
ax[1].fill_between(x, mu-std*3, mu+std*3, alpha=0.5);
fig, ax = plt.subplots(figsize=(6, 4), tight_layout=True)
ax[0].hist(thetas[:, 0])

In [ ]:
plt.close('all'); fig, ax = plt.subplots(figsize=(7, 4))
def update(N, T):
    ax.cla(); ax.set_xlabel('x')
    np.random.seed(0)
    x = np.random.randn(N) # zero mean, unit variance
    mle_mu = np.mean(x)    
    mle_mu_bs = [np.mean(np.random.choice(x, size=len(x), replace=True)) for k in range(T)]
    hist_val, hist_lim, _ = ax.hist(mle_mu_bs, density=True, alpha=0.6)
    t = np.linspace(hist_lim[0], hist_lim[-1], num=200)
    ax.plot(t, gaussian_pdf(t, mu=mle_mu, s=1/np.sqrt(len(x))), 'k-', linewidth=4)  
    ax.scatter(np.mean(x), 0, c='k', s=100, zorder=100)
    display("Empirical confidence interval at 0.95 = [%0.4f, %0.4f]" %(np.sort(mle_mu_bs)[int(0.05*T)], 
                                                                       np.sort(mle_mu_bs)[int(0.95*T)]))    
widgets.interact(update, N=SelSlider_nice(options=[10, 100, 1000, 10000], value=100),
         T=SelSlider_nice(options=[10, 100, 1000, 10000], value=100));

https://www.sciencedirect.com/science/article/pii/S0167715218300737